# EV Stations

In [1]:
# Importing the required modules
from flask import Flask, render_template, jsonify
from flask_pymongo import PyMongo
import pandas as pd
import json

In [ ]:
def stationCounts():
    # Extracting charging stations csv file into dataframe
    charging_stations = "Data/Electric and Alternative Fuel Charging Stations.csv"
    scount_df = pd.read_csv(charging_stations, low_memory=False)

    # Clean Sales dataset by replacing NaNs with 0
    scount_df = scount_df.fillna(0)
    
    # Select only electric chargers in the US
    scount_df = scount_df.loc[(scount_df['Fuel Type Code'] == 'ELEC') & (scount_df['Country'] == 'US')].reset_index(drop=True)

    # Get only public charging stations
    scount_df = scount_df.loc[scount_df['Access Code'] == 'public'].reset_index(drop=True)

    # delete unused columns
    scount_df = scount_df.drop(columns=['Fuel Type Code','Groups With Access Code','EV Other Info','EV Network Web','Geocode Status','Date Last Confirmed','Facility Type','Status Code','Expected Date','Access Days Time','Updated At','Country','Access Code','EV Pricing','EV On-Site Renewable Source','Restricted Access','Intersection Directions','Plus4','Station Phone','Cards Accepted','BD Blends','NG Fill Type Code','NG PSI','ID','Owner Type Code','Federal Agency ID','Federal Agency Name','Hydrogen Status Link','NG Vehicle Class','LPG Primary','E85 Blender Pump','Intersection Directions (French)','Access Days Time (French)','BD Blends (French)','Groups With Access Code (French)','Hydrogen Is Retail','Access Detail Code','Federal Agency Code','CNG Dispenser Num','CNG On-Site Renewable Source','CNG Total Compression Capacity','CNG Storage Capacity','LNG On-Site Renewable Source','E85 Other Ethanol Blends','EV Pricing (French)','LPG Nozzle Types','Hydrogen Pressures','Hydrogen Standards','CNG Fill Type Code','CNG PSI','CNG Vehicle Class','LNG Vehicle Class'])

    # change the date to only the year
    scount_df['Year Opened'] = pd.DatetimeIndex(scount_df['Open Date']).year

    # remove the old date
    scount_df.drop(columns=['Open Date'], inplace=True)

    scount_df.drop(scount_df.columns[[0,1,2,3,4,5,6,7,8,9,10,11]], axis=1, inplace=True)

    year_count = scount_df.groupby('Year Opened').value_counts().reset_index()

    year_count.rename(columns={0 : 'Stations Added'}, inplace=True)

    year_count['Total Stations'] = year_count['Stations Added'].cumsum()
    
    return year_count.to_dict('split')
    
    

In [2]:
def stations():
    # Extracting charging stations csv file into dataframe
    charging_stations = "Data/Electric and Alternative Fuel Charging Stations.csv"
    stations_df = pd.read_csv(charging_stations, low_memory=False)

    # Clean Sales dataset by replacing NaNs with 0
    stations_df = stations_df.fillna(0)
    
    # Select only electric chargers in the US
    stations_df = stations_df.loc[(stations_df['Fuel Type Code'] == 'ELEC') & (stations_df['Country'] == 'US')].reset_index(drop=True)

    # Get only public charging stations
    stations_df = stations_df.loc[stations_df['Access Code'] == 'public'].reset_index(drop=True)

    # delete unused columns
    stations_df = stations_df.drop(columns=['Fuel Type Code','Groups With Access Code','EV Other Info','EV Network Web','Geocode Status','Date Last Confirmed','Facility Type','Status Code','Expected Date','Access Days Time','Updated At','Country','Access Code','EV Pricing','EV On-Site Renewable Source','Restricted Access','Intersection Directions','Plus4','Station Phone','Cards Accepted','BD Blends','NG Fill Type Code','NG PSI','ID','Owner Type Code','Federal Agency ID','Federal Agency Name','Hydrogen Status Link','NG Vehicle Class','LPG Primary','E85 Blender Pump','Intersection Directions (French)','Access Days Time (French)','BD Blends (French)','Groups With Access Code (French)','Hydrogen Is Retail','Access Detail Code','Federal Agency Code','CNG Dispenser Num','CNG On-Site Renewable Source','CNG Total Compression Capacity','CNG Storage Capacity','LNG On-Site Renewable Source','E85 Other Ethanol Blends','EV Pricing (French)','LPG Nozzle Types','Hydrogen Pressures','Hydrogen Standards','CNG Fill Type Code','CNG PSI','CNG Vehicle Class','LNG Vehicle Class'])

    # change the date to only the year
    stations_df['Year Opened'] = pd.DatetimeIndex(stations_df['Open Date']).year

    # remove the old date
    stations_df.drop(columns=['Open Date'], inplace=True)
    
    return stations_df.to_dict('split')

In [3]:
# Create an instance of Flask
app = Flask(__name__)

In [4]:
# Setup connection to mongodb
conn = "mongodb://localhost:27017/ev_data"
mongo = PyMongo(app, uri = conn)

In [5]:
charging_station = stations()

In [ ]:
station_count = stationCounts()

In [8]:
years = []

In [6]:
stations = charging_station['data']

In [9]:
for station in stations:
    years.append(station[12])

In [10]:
mongo.db.stations.update_one({}, {"$set": charging_station}, upsert=True)

In [11]:
result = mongo.db.stations.find()

In [12]:
result_list = list(result)

In [13]:
result

In [14]:
if len(result_list) > 0:
        # Return the first result only and strip off the '_id'
        data = result_list[0] 
        id_to_discard = data.pop('_id', None)
else:
        # Construct an error message
        data = {'Error': 'No data found'} 

In [16]:
data

{'columns': ['Station Name',
  'Street Address',
  'City',
  'State',
  'ZIP',
  'EV Level1 EVSE Num',
  'EV Level2 EVSE Num',
  'EV DC Fast Count',
  'EV Network',
  'Latitude',
  'Longitude',
  'EV Connector Types',
  'Year Opened'],
 'data': [['Los Angeles Convention Center',
   '1201 S Figueroa St',
   'Los Angeles',
   'CA',
   '90015',
   0.0,
   12.0,
   0.0,
   'Non-Networked',
   34.040539,
   -118.271387,
   'J1772',
   1995],
  ['California Air Resources Board',
   '9530 Telstar Ave',
   'El Monte',
   'CA',
   '91731',
   0.0,
   3.0,
   0.0,
   'Non-Networked',
   34.06872,
   -118.064,
   'J1772',
   1996],
  ['Scripps Green Hospital',
   '10666 N Torrey Pines Rd',
   'La Jolla',
   'CA',
   '92037',
   0.0,
   1.0,
   0.0,
   'Non-Networked',
   32.89947,
   -117.243,
   'J1772',
   1997],
  ['Galpin Motors',
   '15421 Roscoe Blvd',
   'Sepulveda',
   'CA',
   '91343',
   0.0,
   2.0,
   0.0,
   'Non-Networked',
   34.221665,
   -118.468371,
   'J1772',
   2012],
  ['Gal